# Exploratory Data Analysis for Plant Disease Classification

Этот ноутбук содержит исследовательский анализ данных (EDA) для датасета PlantVillage. Мы загрузим данные, визуализируем примеры изображений, проверим распределение классов и подготовим данные для обучения.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings('ignore')

## 1. Загрузка данных

Предполагается, что данные находятся в папке `../data/plantvillage` и организованы по классам (каждая подпапка - класс).

In [ ]:
data_dir = '../data/plantvillage'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = dataset.classes
print(f"Найдено классов: {len(class_names)}")
print(f"Классы: {class_names}")
print(f"Всего изображений: {len(dataset)}")

## 2. Распределение классов

In [ ]:
labels = [label for _, label in dataset.samples]

plt.figure(figsize=(10, 5))
sns.countplot(x=labels)
plt.xticks(ticks=range(len(class_names)), labels=class_names, rotation=45)
plt.title('Распределение классов')
plt.xlabel('Класс')
plt.ylabel('Количество изображений')
plt.show()

## 3. Примеры изображений

In [ ]:
def imshow(img):
    img = img.numpy().transpose((1, 2, 0))
    # Восстанавливаем приблизительные значения (без нормализации)
    img = img * 0.5 + 0.5  # если была нормализация в [0,1], здесь можно убрать
    plt.imshow(img)
    plt.axis('off')

# Загружаем по одному изображению из каждого класса
fig, axes = plt.subplots(1, len(class_names), figsize=(15, 5))
for i, class_name in enumerate(class_names):
    # Находим первый индекс с нужной меткой
    idx = next(j for j, (_, lbl) in enumerate(dataset.samples) if lbl == i)
    img, _ = dataset[idx]
    axes[i].imshow(np.transpose(img.numpy(), (1, 2, 0)))
    axes[i].set_title(class_name)
    axes[i].axis('off')
plt.tight_layout()
plt.show()

## 4. Проверка размеров изображений

Убедимся, что все изображения имеют одинаковый размер после ресайза.

In [ ]:
img, label = dataset[0]
print(f"Размер тензора: {img.shape}")

## 5. Разделение на train/val

Для обучения мы будем использовать случайное разделение (80% train, 20% val).

In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"Тренировочная выборка: {len(train_dataset)} изображений")
print(f"Валидационная выборка: {len(val_dataset)} изображений")

## 6. Заключение

Данные загружены, классы сбалансированы (в датасете PlantVillage примерно равное количество здоровых и больных листьев). Изображения приведены к единому размеру. Можно приступать к обучению модели.